# Znakowanie i identyfikacja kodu wygenerowanego przez AI

Temat automatycznej generacji kodu źródłowego przez Sztuczną Inteliencję (AI) jest obszerny i obejmuje różne techniki, modele oraz zastosowania. Przedmiotem naszego zainteresowania na kursie jest natomiast znalezienie sposobu na zrozumienie czy popularne generatory kodu (i ogólnego użycia) tworzą go w specyficzny dla siebie sposób, i jak tak to jaki. Chcemy odpowiedzieć na pytanie czy w dostarczonym kodzie można odnaleźć pewne statystyczne wzorce - czyli, czy AI posiada swój styl mogący go później zidentyfikować jako autora - podobnie do programistów.

---

Naszą pierwszą czynnością było wygenerowanie kilku prostych matematycznych funkcji i algorytmów w języku Python przy użyciu ChatGPT. Już na pierwszy rzut okna dało się zauważyć pewne elementy, które mogłyby odbiegać od *normy*:
- kod nie korzysta możliwości Pythona co do pisania zwięzłych i bardziej złożonych struktur syntaktycznych
- przy każdej operacji pojawia się komentarz zaczynający się od wielkiej litery, a zmienne wykorzystywane mają dokładnie taką nazwę jak w komentarzu

Celem jest znalezienie **statystycznego** potwierdzenia naszej intuicji, oraz znalezienie **ukrytych artefaktów** o ile istnieją - za pomocą różnych metod i narzędzie programistycznych.

Zadanie detekcji, czy dany fragement kodu został napisany przez AI okazał się niezbyt dobrze opisanym w literaturze tematem, a przynajmniej takie odnieślismy wrażenie. Większość artykułów dotyczyło danych w postaci **tekstowej**. Kod oczywiście również jest zapisany w postaci tekstowej, jednak języki programowania ze względu na swoje przeznaczenie różnią się w aspektach gramatycznych i składniowych, które potrafiły być czynnikiem decydującym o decyzji czy badany tekst jest dziełem człowieka czy AI.
Wyżej wymienione przypuszczenia przykuły też uwagę innych badaczy [1](https://arxiv.org/abs/2405.16133), którzy również zwrócili uwagę na dysproporcję w dokumentacji wykrywania wygenerowanych przez AI fragmentów kodu a tekstu i niedostępności datasetów - co było trochę oczekiwane, ponieważ dopiero od w miare niedługiego okresu, rozwiązania AI stały się użytecznym narzędziem a zarazem problemem.

W swoim artykule zaprezentowali metodę wykrywania polegającą na porównaniu przepisywania przez AI kodu przygotowanego przez 1. człowieka i 2. ai. 

![llm rewriting](./assets/llmrewriting.png)

Dzięki tej obserwacji przygotowali dataset z sztucznie wygenerowanymi funkcjami, z którego możemy skorzystać. Jednak zdecydowali się nauczyć model, a my chcemy deterministycznie znaleźć te różnice.

W innym znalezionym badaniu [2] [https://ieeexplore.ieee.org/document/9674263/], badacze zdecydowali się stworzyć algorytm heurystyczny, który uwzględniał analizę programów z repozytoriów.

![heurystyka](./assets/heurystyka.png)

Na jego podobieństwo zbudowaliśmy własny algorytm heurystyczny, który mimo swojej prostoty i małej próbki danych potrafił wskazać na pochodzenie syntetyczne lub nie:

In [ ]:
import os
import ast
import re
from collections import defaultdict


def analyze_code(code, filename):
    results = {"Filename": filename}

    # Keyword Distribution
    keywords = ['if', 'else', 'for', 'while', 'def', 'class', 'try', 'except']
    keyword_distribution = {kw: len(re.findall(r'\b' + kw + r'\b', code)) for kw in keywords}
    results["Keyword Distribution"] = keyword_distribution

    # Naming Conventions
    pascal_case = len(re.findall(r'\b[A-Z][a-z]*[A-Z][a-z]*\b', code))
    snake_case = len(re.findall(r'\b[a-z]+(_[a-z]+)+\b', code))
    results["Naming Conventions"] = {"PascalCase": pascal_case, "snake_case": snake_case}
 
    # Comment Analysis
    comments = re.findall(r'#.*', code)
    average_comment_length = sum(len(comment) for comment in comments) / len(comments) if comments else 0
    overly_detailed_comments = sum(1 for comment in comments if len(comment) > 40)
    results["Comments"] = {
        "Total Comments": len(comments),
        "Average Length": average_comment_length,
        "Overly Detailed Comments": overly_detailed_comments
    }

    # Cyclomatic Complexity
    tree = ast.parse(code)
    complexity = 1
    for node in ast.walk(tree):
        if isinstance(node, (ast.If, ast.For, ast.While, ast.Try, ast.FunctionDef)):
            complexity += 1
    results["Cyclomatic Complexity"] = complexity

    # Code Duplication Detection
    def find_duplicate_functions(tree):
        function_names = defaultdict(list)
        for node in ast.walk(tree):
            if isinstance(node, ast.FunctionDef):
                function_names[node.name].append(ast.get_source_segment(code, node))

        duplicates = {name: func_code for name, func_code in function_names.items() if len(func_code) > 1}
        return duplicates

    duplicates = find_duplicate_functions(tree)
    results["Duplicate Functions"] = {name: len(funcs) for name, funcs in duplicates.items()}

    # Repetitive Patterns Check
    repetitive_patterns = re.findall(r'\b\w+\b', code)
    repetitive_count = len([word for word in repetitive_patterns if repetitive_patterns.count(word) > 3])
    results["Repetitive Patterns"] = repetitive_count

    # Variable & Function Naming Analysis
    overly_descriptive_names = sum(
        1 for name in re.findall(r'\b[a-zA-Z_]{10,}\b', code) if '_' in name
    )
    results["Overly Descriptive Names"] = overly_descriptive_names

    # Simple Logic and Default Values
    default_values = len(re.findall(r'\b=\s*[\'\"\d\[\]\{\}\(\)]', code))
    results["Default Values"] = default_values

    # Exception Handling
    exception_handlers = len(re.findall(r'\btry\b.*?\bexcept\b', code, re.DOTALL))
    results["Exception Handling"] = exception_handlers

    # AI Generated Probability
    ai_score = 0
    total_weight = 9

    ai_score += (complexity < 10) * (1 / total_weight)
    ai_score += (average_comment_length > 15) * (2 / total_weight)
    ai_score += (pascal_case == 0) * (1 / total_weight)
    ai_score += (snake_case > 0) * (1 / total_weight)
    ai_score += (len(duplicates) > 0) * (0.5 / total_weight)
    ai_score += (repetitive_count > 5) * (1 / total_weight)
    ai_score += (overly_descriptive_names > 2) * (1 / total_weight)
    ai_score += (default_values > 2) * (0.5 / total_weight)
    ai_score += (exception_handlers <= 2) * (1 / total_weight)

    ai_probability = ai_score * 100
    results["AI Generated Probability (%)"] = round(ai_probability, 2)

    return results


def analyze_directory(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".py"):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r') as file:
                code = file.read()
                results = analyze_code(code, filename)

                print(f"=== Analysis for {directory}/{filename} ===")
                # for key, value in results.items():
                #     print(f"{key}: {value}")
                print(f"'AI Generated Probability (%): {results['AI Generated Probability (%)']}")
                print("\n")


analyze_directory('code_samples/ai')
analyze_directory('code_samples/human')

=== Analysis for code_samples/ai/binary_search.py ===
'AI Generated Probability (%): 77.78


=== Analysis for code_samples/ai/bubble_sort.py ===
'AI Generated Probability (%): 77.78


=== Analysis for code_samples/ai/factorial.py ===
'AI Generated Probability (%): 77.78


=== Analysis for code_samples/ai/fib.py ===
'AI Generated Probability (%): 88.89


=== Analysis for code_samples/ai/is_prime.py ===
'AI Generated Probability (%): 77.78


=== Analysis for code_samples/ai/palindrome.py ===
'AI Generated Probability (%): 77.78


=== Analysis for code_samples/human/binary_search.py ===
'AI Generated Probability (%): 66.67


=== Analysis for code_samples/human/bubble_sort.py ===
'AI Generated Probability (%): 44.44


=== Analysis for code_samples/human/factorial.py ===
'AI Generated Probability (%): 33.33


=== Analysis for code_samples/human/fib.py ===
'AI Generated Probability (%): 44.44


=== Analysis for code_samples/human/is_prime.py ===
'AI Generated Probability (%): 44.44


=== Ana

Obliczone prawdopodobieństwo przyjmuje zauważalnie wyższe wartości dla syntetycznych kodów. 

---

Kolejnym etapem jest skupienie się na sposobie zakodowania kodu - spróbowaliśmy zawrzeć wzorzec do kodu. Mianowicie, dla wybranego tekstu kodu, w co drugiej linijce dodajemy spację lub tabulator do końca linii. Te znaki będą nam mówić o wartości bitu. 
- spacja = 0
- tab = 1

Znaki są ustawiane w taki sposób, aby odczytując kod z góry do dołu tworzyły nam się bloki bajtowe, które są kodem litery naszego hasła.
Limitacją na razie jest długość kodu i długość hasła, ale to będzie ulepszane. 

In [ ]:
def add_watermark(code, watermark="LABORATORIA"):
    watermark_binary = ''.join(
        ['\t' if bit == '1' else ' ' for bit in ''.join(format(ord(c), '08b') for c in watermark)])
    print("Watermark binary:" + watermark_binary)
    print('Dlugosc watermarku: ' + str(len(watermark)))
    print('Dlugosc binarna watermarku: ' + str(len(watermark_binary)))
    lines = code.splitlines()

    for i in range(1, len(lines), 2):
        if i // 2 < len(watermark_binary):
            lines[i] += watermark_binary[i // 2]

    return '\n'.join(lines)


def extract_watermark(code):
    lines = code.splitlines()

    binary_pattern = []
    for i in range(1, len(lines), 2):
        if lines[i].endswith(" ") or lines[i].endswith("\t"):
            last_char = lines[i][-1]
            binary_pattern.append('1' if last_char == '\t' else '0')

    watermark_text = ''
    for i in range(0, len(binary_pattern), 8):
        byte = ''.join(binary_pattern[i:i + 8])
        if len(byte) == 8:
            watermark_text += chr(int(byte, 2))

    return watermark_text if watermark_text else "Brak znaku wodnego"


with open("../init-code/init.py", "r") as f:
    code_example = f.read()

watermarked_code = add_watermark(code_example)
print("Watermarkowany kod:" + watermarked_code)

detected_watermark = extract_watermark(watermarked_code)
print("Odczytany przykladowy kod:", code_example)

detected_watermark_2 = extract_watermark(watermarked_code)
print("Odczytany watermark:", detected_watermark_2)


In [18]:
''' NA RAZIE NIE
example_ai = 'code_samples/ai/binary_search.py'
with open(example_ai, 'rb') as f:
    content_string = f.readlines()

print(content_string[0][0])
print(content_string[0])
print(format(content_string[0][0], '08b'))
print('--------------------------')

latin_chars_and_nrs = [(65, 122), (48, 57)]

def check_for_unusual_chars(file_bytelines: list):
    unusual_chars = []
    for i, line in enumerate(file_bytelines):
        print("FILE LINE IN BYTES:", line)
        for char_nr in line:
            if not any(usual[0] <= char_nr <= usual[1] for usual in latin_chars_and_nrs):
                unusual_chars.append((i, char_nr))
    return unusual_chars


unusual_chars = check_for_unusual_chars(content_string)
print("UNUSUAL CHARACTERS: ", unusual_chars)
'''
print()